In [1]:
import uproot3
import os
from tqdm.notebook import tqdm
import pandas as pd
import re
import numpy as np

In [2]:
_nsre = re.compile('([0-9]+)')
def natural_sort_key(s):
    return [int(text) if text.isdigit() else text.lower()
            for text in re.split(_nsre, s)]

In [3]:
path = "TAO data"
files = os.listdir(path)
files.sort(key=natural_sort_key)

#targets = pd.DataFrame()
#signal = np.empty([2, 0])
for file_name in tqdm(files):
    file = uproot3.open(path+'/'+file_name)
    
    df = file[file.keys()[0]].pandas.df(flatten=False)
    
    edep_array = []
    edepX_array = []
    edepY_array = []
    edepZ_array = []
    
    hitt_arrays = []
    hitid_arrays = []
    
    for evt in tqdm(range(len(df)), leave=False):
        edep_array.append(df.iloc[evt].GdLSEdep)
        edepX_array.append(df.iloc[evt].GdLSEdepX / 1000.)
        edepY_array.append(df.iloc[evt].GdLSEdepY / 1000.)
        edepZ_array.append(df.iloc[evt].GdLSEdepZ / 1000.)  
                             
        hitt = np.array(df.iloc[evt].SiPMHitT)
        hitid = np.array(df.iloc[evt].SiPMHitID)
        
        hitt_arrays.append(hitt)
        hitid_arrays.append(hitid)
                
    hitt = np.array(hitt_arrays, dtype=object)
    hitid = np.array(hitid_arrays, dtype=object)
    raw_data = np.vstack((hitid, hitt))

    data = pd.DataFrame(data=edep_array)
    data.columns = ['edep']
    data['edepX'] = edepX_array
    data['edepY'] = edepY_array
    data['edepZ'] = edepZ_array

    #targets = targets.append(data)
    #signal = np.hstack((signal, raw_data)) 

    np.savez_compressed(f'train_data/raw/signal_{file_name[:-5]}.npz', a=raw_data)
    data.to_csv(f'train_data/targets/targets_{file_name[:-5]}.csv', index=False)

#np.savez_compressed('TAO data preprocessed/signal.npz', a=signal)
#targets.to_csv(f'TAO data preprocessed/targets.csv', index=False)